### Load data set with features

In [196]:
from create_df import read_df
df, dfc, all_homes, appliance_min, national_average = read_df()

In [197]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

In [232]:
run all_functions.py

/Users/nipunbatra/git/monthly-disaggregation/code/all_functions.py:260: SyntaxWarning: name 'k' is assigned to before global declaration
  global k


In [230]:
run features.py

In [200]:
import json

In [201]:
optimal_dict= json.load(open("optimal_dict.json","r"))

In [209]:
criterion_function(df, dfc, all_homes, appliance_min, national_average,
                       appliance="hvac", feature=optimal_dict['hvac']['All']['f'],k=4,
                       train_outlier=True, test_outlier=True, outlier_features=optimal_dict['hvac']['All']['f'], outlier_fraction=0.1,
                       metric="median")

83.117444159888763

In [ ]:
monthly_plus_static_features

In [ ]:
seq_forw_select(df, dfc, all_homes, appliance_min, national_average,
                        appliance="hvac", features=all_features[:].tolist(),NUM_NEIGHBOURS=2,
                        train_outlier=True, test_outlier=True, outlier_features=['aggregate_8'], outlier_fraction=0.1,
                        metric="mean",
                        max_k=3, criterion_func=criterion_function, print_steps=True)

In [224]:
a,b,c=create_predictions(df, dfc, all_homes, appliance_min, national_average, appliance="hvac", feature=['aggregate_1'],k=2,
                       train_outlier=True, test_outlier=False, outlier_features=None, outlier_fraction=0.1)

hvac
['aggregate_1']
33


In [227]:
c.fit(a,b)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=2, p=2,
          weights='uniform')

In [228]:
c.predict(a)

array([ 309.95834351,  154.66885376,  213.17845535,  323.51126099,
        218.49311829,  173.08065033,  309.95834351,  298.95755005,
        297.45372009,  298.95755005,  202.71150207,  266.0319519 ,
        329.93756103,  326.75569153,  266.0319519 ,  329.93756103,
        209.85571289,  165.06554413,  193.99334717,  202.71150207,
        277.00565338,  218.49311829,  124.00513458,  154.66885376,
        165.06554413,  204.01132584,  193.99334717,  204.01132584])

In [229]:
b

array([ 275.581604  ,  145.88465881,   78.73975372,  371.44091797,
        185.52867127,  112.34846497,  344.33508301,  307.73428345,
        280.61193848,  290.18081665,  171.61016846,  184.44674683,
        314.29550171,  307.93176269,  347.61715698,  345.57962036,
        231.59886169,  199.98657227,  188.11256409,  233.81283569,
        246.07954407,  251.45756531,   84.55722046,  163.45304871,
        130.14451599,   97.66996002,  199.87413025,  310.35269165])

In [114]:
temp = create_predictions(df, dfc, all_homes, appliance_min, national_average, appliance="hvac",feature=['seasonal_energy_8'],
                          k=3, test_outlier=False, outlier_features=['seasonal_energy_8'])
errors = {}
if appliance =="hvac":
    start_month, end_month = 5, 11
else:
    start_month, end_month = 1, 13
for i in range(start_month, end_month):
    errors[i] = percentage_error(temp[i]["gt"], temp[i]["pred"])
error_df = pd.DataFrame(errors)
accur_df = 100-error_df
accur_df[accur_df<0]=0

In [115]:
print accur_df.mean().mean()
print accur_df.median().mean()


72.9696966674
79.7508667304


In [100]:
appliance='hvac'

In [104]:
accur_df.mean().mean()

60.526389859059002

In [ ]:
find_optimal_features(df, dfc, all_homes, appliance_min, national_average, ["hvac"], {"t":['aggregate_1','aggregate_2','aggregate_3']},
                          NUM_NEIGHBOURS_MAX=2, F_length_max=2, metric="median",
                          train_outlier=True, test_outlier=True, outlier_features=['seasonal_energy_8'], outlier_fraction=0.1)